In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Replace 'your_folder_path' and 'your_file_name.csv' with the actual path and file name
file_path_2025 = '/content/drive/My Drive/Bridge Pledge/csv_files_inputs/app_data_2025.csv'
file_path_2024 = '/content/drive/My Drive/Bridge Pledge/csv_files_inputs/app_data_2024.csv'
file_path_comm_2025 = '/content/drive/My Drive/Bridge Pledge/csv_files_inputs/app_data_communications_2025.csv'

#read in files
df_2025 = pd.read_csv(file_path_2025)
df_2024 = pd.read_csv(file_path_2024)
df_comm_2025 = pd.read_csv(file_path_comm_2025)

In [ ]:
#communication data analysis
df_comm_filtered = df_comm_2025[['bioguide_id', 'first_name','last_name','year','attack_personal','outcome_bipartisanship']]

In [ ]:
#only include certain columns
df_2025_filtered = df_2025[['bioguide_id','full_name', 'communication_count','communication_outcome_bipartisanship_sum',
              'communication_outcome_bipartisanship_mean','communication_attack_personal_sum', 'communication_attack_personal_mean']]

df_2024_filtered = df_2024[['bioguide_id','full_name', 'communication_count','communication_outcome_bipartisanship_sum',
              'communication_outcome_bipartisanship_mean','communication_attack_personal_sum', 'communication_attack_personal_mean']]


In [ ]:
#clean df_2024 data so names are matching df_2025 data

# def clean_full_name(name):
#     if pd.isna(name):
#         return name

#     # # 1. Replace 'chuck' with 'charles' (case-insensitive)
#     # name = re.sub(r'\bchuck\b', 'Charles', name, flags=re.IGNORECASE)

#     # # 1. Replace 'Alex' with 'Alejandro' (case-insensitive)
#     # name = re.sub(r'\bAlex\b', 'Alejandro', name, flags=re.IGNORECASE)

#     # # 1. Replace 'Jack' with 'John' (case-insensitive)
#     # name = re.sub(r'\bjack\b', 'John', name, flags=re.IGNORECASE)

#     # # 1. Replace 'Kat' with 'Katherine' (case-insensitive)
#     # name = re.sub(r'\bkat\b', 'Katherine', name, flags=re.IGNORECASE)

#     # # 1. Replace 'Tim' with 'Timothy' (case-insensitive)
#     # name = re.sub(r'\btim\b', 'Timothy', name, flags=re.IGNORECASE)

#     # 2. Remove anything in single or double quotes
#     name = re.sub(r'["\'].*?["\']', '', name)

#     # 3. Remove middle initials (e.g., A.)
#     name = re.sub(r'(?<!^)\b[A-Z]\.(?=\s)', '', name)

#     # 4. Remove suffixes like Jr., III, etc.
#     name = re.sub(r'[, ]*\b(jr\.?|sr\.?|ii|iii|iv|v|vi|vii|viii|ix|x)\b\.?', '', name, flags=re.IGNORECASE)

#     # 5. Normalize whitespace
#     name = re.sub(r'\s+', ' ', name).strip()

#     # 6. Handle name logic
#     parts = name.split()

#     # If name starts with an initial and has 3 parts, remove middle name
#     if len(parts) == 3 and re.fullmatch(r'[A-Z]\.', parts[0]):
#         parts.pop(1)  # Remove middle name, keep initial and last name

#     # Otherwise, remove middle name unless it's 'Van'
#     elif len(parts) >= 3:
#         if parts[1].lower() != 'van':
#             parts.pop(1)


#     return ' '.join(parts)

# # Step 2: Sync names by matching last names and updating first names
# def sync_first_names(df1, df2):
#     # Clean both DataFrames
#     df1['clean_name'] = df1['full_name'].apply(clean_full_name)
#     df2['clean_name'] = df2['full_name'].apply(clean_full_name)

#     # Split cleaned names into first and last
#     df1[['first_name', 'last_name']] = df1['clean_name'].str.extract(r'^(\S+)\s+(\S+)$')
#     df2[['first_name', 'last_name']] = df2['clean_name'].str.extract(r'^(\S+)\s+(\S+)$')

#     # Merge on last name to compare first names
#     merged = df1.merge(df2[['first_name', 'last_name']], on='last_name', how='left', suffixes=('_df1', '_df2'))

#     # Update first names in df1 if last name matches but first name differs
#     merged['first_name_final'] = merged.apply(
#         lambda row: row['first_name_df2']
#         if pd.notna(row['first_name_df2']) and row['first_name_df1'] != row['first_name_df2']
#         else row['first_name_df1'],
#         axis=1
#     )

#     # Rebuild the full name
#     merged['full_name_updated'] = merged['first_name_final'] + ' ' + merged['last_name']

#     # Return df1 with updated names
#     df1['full_name'] = merged['full_name_updated']
#     return df1

# # df_2024_copy = df_2024_filtered.copy()
# df_2024_filtered = sync_first_names(df_2024_filtered, df_2025_filtered)

In [ ]:
#merge df_2024 and df_2025 on bio_guide and full_name, and keep all columns, only keep names that exist in both df_2024 and df_2025
merged_df = pd.merge(df_2024_filtered, df_2025_filtered, on=['bioguide_id'], how='inner', suffixes=('_2024', '_2025'))

#calculate differences in 2024 and 2025 data to just isolate 2025 instances
merged_df['communication_count_isolated_2025'] = merged_df['communication_count_2025'] - merged_df['communication_count_2024']
merged_df['communication_outcome_bipartisanship_sum_isolated_2025'] = merged_df['communication_outcome_bipartisanship_sum_2025'] - merged_df['communication_outcome_bipartisanship_sum_2024']
merged_df['communication_attack_personal_sum_isolated_2025'] = merged_df['communication_attack_personal_sum_2025'] - merged_df['communication_attack_personal_sum_2024']

#only include necessary columsn
df_2024_2025 = merged_df[['bioguide_id', 'full_name_2025', 'communication_count_isolated_2025', 'communication_outcome_bipartisanship_sum_2025', 'communication_outcome_bipartisanship_sum_isolated_2025','communication_attack_personal_sum_2025', 'communication_attack_personal_sum_isolated_2025', 'communication_outcome_bipartisanship_mean_2025','communication_attack_personal_mean_2025']].sort_values(by='communication_count_isolated_2025', ascending=False)

#remove all _isolated_2025 or _2025 suffixes
# suffixes = ['_isolated_2025', '_2025']
# df_2024_2025.columns = [
#     next((col[:-len(suffix)] for suffix in suffixes if col.endswith(suffix)), col)
#     for col in df_2024_2025.columns]

df_2024_2025


,bioguide_id,full_name_2025,communication_count_isolated_2025,communication_outcome_bipartisanship_sum_2025,communication_outcome_bipartisanship_sum_isolated_2025,communication_attack_personal_sum_2025,communication_attack_personal_sum_isolated_2025,communication_outcome_bipartisanship_mean_2025,communication_attack_personal_mean_2025
355,D000563,Richard Durbin,6233.0,1687.0,714.0,359.0,279.0,9.11,1.94
62,S000148,Charles Schumer,4547.0,1980.0,561.0,295.0,31.0,12.88,1.92
321,M000194,Nancy Mace,4063.0,265.0,86.0,597.0,406.0,3.17,7.15
123,W000817,Elizabeth Warren,3966.0,495.0,344.0,295.0,223.0,6.01,3.58
437,T000278,Tommy Tuberville,3514.0,348.0,188.0,295.0,16.0,3.42,2.90
...,...,...,...,...,...,...,...,...,...
403,L000562,Stephen Lynch,161.0,63.0,24.0,6.0,5.0,10.66,1.02
253,U000040,Lauren Underwood,159.0,47.0,10.0,3.0,-8.0,4.67,0.30
98,S001157,David Scott,92.0,34.0,9.0,0.0,0.0,6.50,0.00
386,D000616,Scott DesJarlais,70.0,7.0,4.0,9.0,-2.0,2.72,3.50


In [ ]:
#count how many communication_attack_personal_sum_isolated are negative numbers
count = (df_2024_2025['communication_count_isolated_2025'] < 0).sum()
print(count)

#print only rows that have negative numbers
df_2024_2025[df_2024_2025['communication_attack_personal_sum_isolated_2025'] < 0]


0


,bioguide_id,full_name_2025,communication_count_isolated_2025,communication_outcome_bipartisanship_sum_2025,communication_outcome_bipartisanship_sum_isolated_2025,communication_attack_personal_sum_2025,communication_attack_personal_sum_isolated_2025,communication_outcome_bipartisanship_mean_2025,communication_attack_personal_mean_2025
240,B001319,Katie Britt,3032.0,665.0,480.0,49.0,-6.0,13.63,1.00
220,T000250,John Thune,2934.0,391.0,209.0,44.0,-60.0,5.24,0.59
347,F000446,Randy Feenstra,2687.0,432.0,176.0,20.0,-18.0,5.06,0.24
207,B001261,John Barrasso,2354.0,220.0,152.0,110.0,-35.0,4.05,2.02
397,C001047,Shelley Capito,2281.0,831.0,302.0,40.0,-18.0,9.27,0.45
...,...,...,...,...,...,...,...,...,...
89,D000096,Danny Davis,225.0,54.0,32.0,1.0,-1.0,6.19,0.11
338,A000371,Pete Aguilar,209.0,68.0,18.0,4.0,-10.0,6.94,0.41
217,C001051,John Carter,200.0,31.0,8.0,2.0,-5.0,1.54,0.10
253,U000040,Lauren Underwood,159.0,47.0,10.0,3.0,-8.0,4.67,0.30


In [ ]:
#create another dataframe that only includes people who are elected in 2025 and did not exist in 2024
merged_df_new = pd.merge(df_2025_filtered, df_2024_filtered, on=['bioguide_id'], how='left', indicator=True)

# Filter for rows only in df_2025
df_2025_only = merged_df_new[merged_df_new['_merge'] == 'left_only']

# Get a list of columns to remove
columns_to_remove = [col for col in df_2025_only.columns if col.endswith(('_merge', '_y'))]

# Remove the columns
df_2025_only = df_2025_only.drop(columns=columns_to_remove)

#remove all _x suffixes
df_2025_only.columns = df_2025_only.columns.str.rstrip('_x')

#concat df_2025 and df_2024_2025
df_final = pd.concat([df_2024_2025, df_2025_only])

#remove mean columns
df_final = df_final.drop(columns=['communication_outcome_bipartisanship_mean', 'communication_attack_personal_mean'])
df_final

,bioguide_id,full_name_2025,communication_count_isolated_2025,communication_outcome_bipartisanship_sum_2025,communication_outcome_bipartisanship_sum_isolated_2025,communication_attack_personal_sum_2025,communication_attack_personal_sum_isolated_2025,communication_outcome_bipartisanship_mean_2025,communication_attack_personal_mean_2025,full_name,communication_count,communication_outcome_bipartisanship_sum,communication_attack_personal_sum
355,D000563,Richard Durbin,6233.0,1687.0,714.0,359.0,279.0,9.11,1.94,NaN,NaN,NaN,NaN
62,S000148,Charles Schumer,4547.0,1980.0,561.0,295.0,31.0,12.88,1.92,NaN,NaN,NaN,NaN
321,M000194,Nancy Mace,4063.0,265.0,86.0,597.0,406.0,3.17,7.15,NaN,NaN,NaN,NaN
123,W000817,Elizabeth Warren,3966.0,495.0,344.0,295.0,223.0,6.01,3.58,NaN,NaN,NaN,NaN
437,T000278,Tommy Tuberville,3514.0,348.0,188.0,295.0,16.0,3.42,2.90,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,R000621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Emily Randall,18.0,1.0,0.0
533,M001235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riley Moore,4.0,0.0,0.0
534,J000312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jim Justice,97.0,0.0,0.0
535,H001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jon Husted,NaN,NaN,NaN


In [ ]:
df_2025_only

In [ ]:
#calculate percentages
df_final['pct_bipartisanship'] = df_final['communication_outcome_bipartisanship_sum'] / df_final['communication_count']*100
df_final['pct_personal'] = df_final['communication_attack_personal_sum'] / df_final['communication_count']*100
df_final

,bioguide_id,full_name_2025,communication_count_isolated_2025,communication_outcome_bipartisanship_sum_2025,communication_outcome_bipartisanship_sum_isolated_2025,communication_attack_personal_sum_2025,communication_attack_personal_sum_isolated_2025,communication_outcome_bipartisanship_mean_2025,communication_attack_personal_mean_2025,full_name,communication_count,communication_outcome_bipartisanship_sum,communication_attack_personal_sum,pct_bipartisanship,pct_personal
461,K000402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Timothy Kennedy,1294.0,48.0,14.0,3.709428,1.081917
460,S001201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thomas Suozzi,1144.0,227.0,1.0,19.842657,0.087413
462,F000480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vince Fong,575.0,29.0,16.0,5.043478,2.782609
463,R000619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Michael Rulli,361.0,1.0,57.0,0.277008,15.789474
489,H001099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mike Haridopolos,275.0,11.0,3.0,4.000000,1.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,H001098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abraham Hamadeh,NaN,NaN,NaN,NaN,NaN
497,B001321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom Barrett,NaN,NaN,NaN,NaN,NaN
498,M001237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kristen McDonald Rivet,NaN,NaN,NaN,NaN,NaN
535,H001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jon Husted,NaN,NaN,NaN,NaN,NaN


In [ ]:
#sort df_final from largest to smallest communication_count
df_final = df_final.sort_values(by='bioguide_id', ascending=True)
df_final

,bioguide_id,full_name_2025,communication_count_isolated_2025,communication_outcome_bipartisanship_sum_2025,communication_outcome_bipartisanship_sum_isolated_2025,communication_attack_personal_sum_2025,communication_attack_personal_sum_isolated_2025,communication_outcome_bipartisanship_mean_2025,communication_attack_personal_mean_2025,full_name,communication_count,communication_outcome_bipartisanship_sum,communication_attack_personal_sum,pct_bipartisanship,pct_personal
362,A000055,Robert Aderholt,359.0,26.0,16.0,8.0,3.0,2.80,0.86,NaN,NaN,NaN,NaN,NaN,NaN
168,A000148,Jake Auchincloss,1257.0,363.0,196.0,48.0,37.0,12.31,1.63,NaN,NaN,NaN,NaN,NaN,NaN
278,A000369,Mark Amodei,387.0,85.0,39.0,7.0,2.0,6.25,0.51,NaN,NaN,NaN,NaN,NaN,NaN
8,A000370,Alma Adams,496.0,88.0,28.0,6.0,-1.0,3.18,0.22,NaN,NaN,NaN,NaN,NaN,NaN
338,A000371,Pete Aguilar,209.0,68.0,18.0,4.0,-10.0,6.94,0.41,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,W000829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tony Wied,143.0,6.0,0.0,4.195804,0.0
480,W000830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,George Whitesides,10.0,0.0,0.0,0.000000,0.0
432,Y000064,Todd Young,1377.0,728.0,277.0,6.0,-1.0,14.24,0.12,NaN,NaN,NaN,NaN,NaN,NaN
378,Y000067,Rudy Yakym,1134.0,132.0,37.0,154.0,64.0,4.32,5.04,NaN,NaN,NaN,NaN,NaN,NaN
